In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import os

In [3]:
os.chdir('/Users/Sriram/Desktop/DePaul/Work - LIDC (Smart CAD)/LIDC')

In [8]:
data = np.loadtxt('./labels.txt', delimiter='\t', skiprows=1, dtype='int64')

In [7]:
#Choosing a single label based on mode, in case of tie, choose the first occurence
newLabels = np.zeros(shape=[data.shape[0]])
for i in range(data.shape[0]):
    bins = np.bincount(data[i,:])
    newLabels[i] = np.argmax(bins)

In [15]:
df = pd.read_csv('./LIDC_REU2015.csv', index_col=0)
df_sub = df.iloc[:,10:75]

In [16]:
minmax = lambda x: (x-x.min())/(x.max()-x.min())
df_norm = df_sub.apply(minmax, 0)

In [17]:
df_norm.to_csv("./LIDCfeatureData.csv", header=False, index=False)

In [18]:
newLabels = pd.DataFrame(data)
newLabels.to_csv('./LIDCLabels.csv', header=False, index=False)

In [19]:
from sklearn.cross_validation import train_test_split

In [20]:
train, test, train_Labels, test_Labels = train_test_split(df_norm, newLabels, test_size = 0.33)
train.to_csv("./LIDCtrain.csv", header=False, index=False)
test.to_csv("./LIDCtest.csv", header=False, index=False)
train_Labels.to_csv("./LIDCtrianLabels.csv", header=False, index=False)
test_Labels.to_csv("./LIDCtestLabels.csv", header=False, index=False)

In [21]:
def SDMat(trainlabels = train_Labels):
    labels = train_Labels.values.flatten()
    global S; global D
    S = np.zeros(shape=[labels.shape[0],labels.shape[0]])
    D = np.ones(shape=[labels.shape[0],labels.shape[0]])
    for i in range(len(labels)):
        for j in range(i,len(labels)):
            if labels[i] == labels[j]:
                S[i,j] = 1; S[j,i] = 1
                D[i,j] = 0; D[j,i] = 0
    S = pd.DataFrame(S); D = pd.DataFrame(D)

In [22]:
SDMat()
S.to_csv('./LIDCSMatrix.csv', header=False, index=False); D.to_csv('./LIDCDMatrix.csv', header=False, index=False)

In [24]:
from sklearn import neighbors

predLabels5 = np.loadtxt('./5nn.csv', delimiter=',')
nn = 5
kNN_nW = neighbors.KNeighborsClassifier(n_neighbors=nn)
kNN_nW.fit(train, train_Labels)
predictions5 = kNN_nW.predict(test)

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


In [25]:
test_Labels.values.flatten()

array([2, 2, 5, 2, 3, 2, 3, 2, 3, 3, 5, 3, 4, 3, 5, 5, 3, 4, 1, 3, 3, 2, 3,
       2, 2, 1, 5, 3, 1, 3, 1, 3, 3, 3, 1, 3, 3, 3, 3, 2, 1, 3, 4, 3, 3, 1,
       3, 1, 4, 1, 2, 4, 3, 3, 4, 3, 5, 2, 3, 5, 3, 3, 3, 1, 5, 3, 4, 5, 2,
       3, 5, 3, 4, 2, 2, 4, 3, 2, 1, 3, 3, 1, 4, 3, 4, 1, 3, 2, 3, 1, 3, 4,
       2, 1, 5, 2, 3, 3, 2, 3, 1, 1, 5, 1, 5, 3, 3, 1, 3, 3, 5, 4, 4, 3, 4,
       3, 5, 1, 2, 4, 3, 4, 3, 2, 2, 3, 4, 4, 3, 3, 1, 3, 4, 1, 2, 4, 3, 2,
       3, 3, 3, 1, 3, 2, 3, 1, 3, 5, 3, 1, 2, 4, 3, 4, 5, 3, 3, 2, 3, 2, 2,
       3, 3, 2, 2, 4, 1, 1, 4, 3, 5, 3, 3, 5, 5, 2, 3, 3, 4, 3, 2, 3, 3, 4,
       2, 3, 3, 4, 5, 2, 4, 4, 5, 4, 2, 2, 1, 3, 4, 5, 3, 5, 3, 3, 3, 3, 4,
       5, 2, 3, 1, 2, 3, 3, 3, 3, 2, 2, 1, 3, 3, 3, 1, 4, 5, 3, 3, 3, 4, 3,
       1, 4, 3, 3, 2, 2, 3, 2, 3, 3, 2, 2, 1, 1, 1, 3, 5, 4, 3, 3, 2, 5, 4,
       3, 3, 4, 3, 4, 4, 2, 2, 3, 2, 1, 3, 3, 3])

In [36]:
nn = 5

kNN_nW = neighbors.KNeighborsClassifier(n_neighbors=nn)
kNN_nW.fit(train, train_Labels)
predictions5 = kNN_nW.predict(test)

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


In [38]:
sum(predictions5.flatten() == test_Labels.values.flatten())/len(predictions5)

0.29213483146067415

In [41]:
sum(predLabels5.flatten() == test_Labels.values.flatten())/len(predLabels5)

0.27715355805243447

In [182]:
predLabels3 = np.loadtxt('./predLabels3.csv', delimiter=','); predLabels.shape
nn = 3

kNN_nW = neighbors.KNeighborsClassifier(n_neighbors=nn)
kNN_nW.fit(train, train_Labels)
predictions3 = kNN_nW.predict(test)

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


In [177]:
sum(predictions3.flatten() == test_Labels.flatten())/len(predLabels)

0.43445692883895132

In [184]:
sum(predLabels3.flatten() == test_Labels.flatten())/len(predLabels)

0.43820224719101125

In [188]:
predLabels7 = np.loadtxt('./predLabels7.csv', delimiter=','); predLabels.shape
nn = 7

kNN_nW = neighbors.KNeighborsClassifier(n_neighbors=nn)
kNN_nW.fit(train, train_Labels)
predictions7 = kNN_nW.predict(test)

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


In [190]:
sum(predictions7.flatten() == test_Labels.flatten())/len(predLabels)

0.50187265917602997

In [189]:
sum(predLabels7.flatten() == test_Labels.flatten())/len(predLabels)

0.4943820224719101

In [220]:
predLabels3 = np.loadtxt('./predLabelsAll3.csv', delimiter=','); predLabels.shape
nn = 3

kNN_nW = neighbors.KNeighborsClassifier(n_neighbors=nn)
kNN_nW.fit(train, train_Labels)
predictions3 = kNN_nW.predict(test)

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


In [217]:
sum(predictions3 == test_Labels.values.flatten())/len(predLabels3)

0.4044943820224719

In [221]:
sum(predLabels3 == test_Labels.values.flatten())/len(predLabels3)

0.38951310861423222

In [222]:
predLabels5 = np.loadtxt('./predLabelsAll5.csv', delimiter=','); predLabels.shape
nn = 5

kNN_nW = neighbors.KNeighborsClassifier(n_neighbors=nn)
kNN_nW.fit(train, train_Labels)
predictions3 = kNN_nW.predict(test)

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


In [223]:
sum(predictions5 == test_Labels.values.flatten())/len(predLabels5)

0.43445692883895132

In [226]:
sum(predLabels5 == test_Labels.values.flatten())/len(predLabels5)

0.44569288389513106

In [231]:
predLabels7 = np.loadtxt('./predLabelsAll7.csv', delimiter=','); predLabels.shape
nn = 7

kNN_nW = neighbors.KNeighborsClassifier(n_neighbors=nn)
kNN_nW.fit(train, train_Labels)
predictions7 = kNN_nW.predict(test)

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


In [229]:
sum(predictions7 == test_Labels.values.flatten())/len(predLabels5)

0.49063670411985016

In [232]:
sum(predLabels7 == test_Labels.values.flatten())/len(predLabels7)

0.47565543071161048